In [21]:
from natasha import (
    Segmenter,
    MorphVocab,
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    NamesExtractor,
    Doc
)
from rdflib import Graph, Namespace, URIRef
import import_ipynb
import FileWork

In [22]:
segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

names_extractor = NamesExtractor(morph_vocab)

In [23]:
text_IT = open(r'C:\Users\Юлия\OneDrive\Документы\Универ\3 курс\8 триместр\Курсовая\Semantic annotation\files\text_IT.txt', encoding='UTF-8').read()
text_Friends = open(r'C:\Users\Юлия\OneDrive\Документы\Универ\3 курс\8 триместр\Курсовая\Semantic annotation\files\text_Friends.txt', encoding='UTF-8').read()

In [24]:
# db_path = r'C:\Users\Юлия\OneDrive\Документы\Универ\3 курс\8 триместр\Курсовая\Semantic annotation\files\ruwordnet-2021.db'
# FileWork.convert_db_to_owl(db_path)

In [25]:
ontology_graph = Graph()
ontology_graph.parse(r'C:\Users\Юлия\OneDrive\Документы\Универ\3 курс\8 триместр\Курсовая\Semantic annotation\files\my_ontologies.owl')

<Graph identifier=Nfbe9117dd8574e498d6431be722292cc (<class 'rdflib.graph.Graph'>)>

In [26]:
doc_IT = Doc(text_IT)
doc_Friends = Doc(text_Friends)

In [27]:
doc_IT.segment(segmenter)
doc_Friends.segment(segmenter)

In [28]:
doc_IT.tag_morph(morph_tagger)
doc_Friends.tag_morph(morph_tagger)

In [29]:
doc_IT.parse_syntax(syntax_parser)
doc_Friends.parse_syntax(syntax_parser)

In [30]:
for token in doc_IT.tokens:
    token.lemmatize(morph_vocab)

for token in doc_Friends.tokens:
    token.lemmatize(morph_vocab)

lemmas_doc_IT = []
lemmas_doc_Friends = []

for token in doc_IT.tokens:
    if token.pos not in {'PUNCT', 'PRED', 'CONJ', 'PART', 'CCONJ', 'SCONJ', 'ADP', 'PRON', 'DET', 'AUX'}:
        lemmas_doc_IT.append(token.lemma.upper())

for token in doc_Friends.tokens:
    if token.pos not in {'PUNCT', 'PRED', 'CONJ', 'PART', 'CCONJ', 'SCONJ', 'ADP', 'PRON', 'DET', 'AUX'}:
        lemmas_doc_Friends.append(token.lemma.upper())

print(lemmas_doc_IT)
print(lemmas_doc_Friends)

['КОМПАНИЯ', 'РЕШАТЬ', 'МНОЖЕСТВО', 'ПРОБЛЕМА', 'ИТ-СФЕРА', 'ОБЪЕДИНЯТЬ', 'ЭКСПЕРТ', 'СОЗДАНИЕ', 'ИННОВАЦИОННЫЙ', 'РЕШЕНИЕ', 'ВАЖНЫЙ', 'СИНХРОНИЗИРОВАТЬ', 'УСИЛИЕ', 'КОМАНДА', 'ДОСТИГНУТЬ', 'ПОСТАВИТЬ', 'ЦЕЛЬ', 'СОТРУДНИК', 'КЛЮЧЕВОЙ', 'ЗВЕНО', 'ВНОСИТЬ', 'ВКЛАД', 'УСПЕХ', 'РАБОТА', 'КОМПАНИЯ', 'ТРЕБОВАТЬ', 'ТЕХНИЧЕСКИЙ', 'НАВЫК', 'УМЕНИЕ', 'РАБОТАТЬ', 'КОЛЛЕКТИВ', 'ВМЕСТЕ', 'СОЗДАВАТЬ', 'ПРОДУКТ', 'МЕНЯТЬ', 'МИР', 'КОМПАНИЯ', 'РАБОЧИЙ', 'МЕСТО', 'СООБЩЕСТВО', 'ЕДИНОМЫШЛЕННИК', 'СТРЕМИТЬСЯ', 'СОВЕРШЕНСТВО', 'ИТ']
['КОМПАНИЯ', 'ДРУГ', 'ВСТРЕЧА', 'ПРАЗДНИК', 'ДЕЛИТЬСЯ', 'РАДОСТЬ', 'СМЕХ', 'ИСТОРИЯ', 'ВМЕСТЕ', 'ПРОВОДИТЬ', 'ВРЕМЯ', 'ПРИРОДА', 'ГОТОВИТЬ', 'ВКУСНЫЙ', 'БЛЮДО', 'УСТРАИВАТЬ', 'ПИКНИК', 'ПРОДУКТ', 'ВЫБИРАТЬ', 'ЛЮБОВЬ', 'СТАНОВИТЬСЯ', 'ЧАСТЬ', 'ОБЩИЙ', 'СТОЛ', 'ФРУКТ', 'СЫР', 'ВНОСИТЬ', 'ВКЛАД', 'АТМОСФЕРА', 'ДРУЖБА', 'КРЕПНУТЬ', 'ВОСПОМИНАНИЕ', 'СТАНОВИТЬСЯ', 'ЯРЧЕ', 'СКОЛЬКО', 'ВРЕМЯ', 'ПРОЙТИ', 'ВСТРЕЧА', 'ВСЕГДА', 'ЗАПОМИНАТЬСЯ', 'ХОРОШИЙ', 'МОМЕНТ']


In [31]:
def get_ontologies(lemmas):
    ontologies_word = {}
    for word in lemmas:
        ontologies_word[word] = []
        synsets = ontology_graph.query(f"""
            SELECT ?synset_id
            WHERE {{
                ?sense rdf:type ns1:sense ;
                    ns1:name ?name ;
                    ns1:lemma ?lemma ;
                    ns1:synset_id ?synset_id .
                
                FILTER(CONTAINS(?lemma, "{word}"))
            }}
        """)
        for row in synsets:
            synset_id = row['synset_id']

            definitions = ontology_graph.query(f"""
                SELECT ?definition
                WHERE {{
                    ?synset rdf:type ns1:synset ;
                            ns1:id "{synset_id}" ;
                            ns1:definition ?definition .
                }}
            """)

            for d in definitions:
                definition = str(d['definition'])
                if definition not in ontologies_word[word]:
                    ontologies_word[word].append(definition)
                    
    return ontologies_word

In [32]:
ontologies_doc_IT = get_ontologies(lemmas_doc_IT)
ontologies_doc_IT

{'КОМПАНИЯ': ['группа лиц, проводящих время совместно',
  'компания, осуществляющая свою деятельность в области информационных технологий'],
 'РЕШАТЬ': ['обдумывать ситуацию, формулируя определённый вывод, делая заключение или вырабатывая план какого-то действия'],
 'МНОЖЕСТВО': [],
 'ПРОБЛЕМА': ['сложный теоретический или практический вопрос, требующий решения, исследования'],
 'ИТ-СФЕРА': [],
 'ОБЪЕДИНЯТЬ': [],
 'ЭКСПЕРТ': ['высококлассный специалист в какой-либо предметной области'],
 'СОЗДАНИЕ': ['создание, творение, плод творчества художника'],
 'ИННОВАЦИОННЫЙ': [],
 'РЕШЕНИЕ': ['обдумывать ситуацию, формулируя определённый вывод, делая заключение или вырабатывая план какого-то действия',
  'обдумав вопрос или исследовав проблему, найти нужный ответ'],
 'ВАЖНЫЙ': [],
 'СИНХРОНИЗИРОВАТЬ': ['приведение к синхронизму двух или нескольких периодически изменяющихся явлений или процессов'],
 'УСИЛИЕ': ['напряжение физических или умственных сил'],
 'КОМАНДА': ['всякая группа лиц, ватага, 

In [33]:
ontologies_doc_Friends = get_ontologies(lemmas_doc_Friends)
ontologies_doc_Friends

{'КОМПАНИЯ': ['группа лиц, проводящих время совместно',
  'компания, осуществляющая свою деятельность в области информационных технологий'],
 'ДРУГ': ['тот, с кем поддерживают близкие, приятельские отношения; друг, приятель, товарищ'],
 'ВСТРЕЧА': ['собрание, устраиваемое с целью знакомства, обсуждения чего-либо и\xa0т.\xa0п'],
 'ПРАЗДНИК': ['официально отмечаемый торжественный день, связанный с годовщиной радостного события или церковной традицией'],
 'ДЕЛИТЬСЯ': ['рассказывая, сообщая кому-либо о чём-либо, уделять кому-либо от своих знаний, сведений',
  'доставить радость кому-либо, вызвать сильное положительное чувство в ком-либо'],
 'РАДОСТЬ': ['доставить радость кому-либо, вызвать сильное положительное чувство в ком-либо'],
 'СМЕХ': ['характерные прерывистые горловые звуки, вызываемые короткими выдыхательными движениями при проявлении веселья, радости, удовольствия и т. п.'],
 'ИСТОРИЯ': ['наука, изучающая факты, тенденции и закономерности развития общества',
  'рассказ, повествов

In [34]:
triples_doc_IT = FileWork.get_triples(text_IT)
triples_doc_Friends = FileWork.get_triples(text_Friends)

print(triples_doc_IT)

[['компания', 'решать', 'множество'], ['решать', 'множество', 'проблема'], ['множество', 'проблема', 'ит-сфера'], ['проблема', 'ит-сфера', 'объединять'], ['ит-сфера', 'объединять', 'эксперт'], ['объединять', 'эксперт', 'создание'], ['эксперт', 'создание', 'инновационный'], ['создание', 'инновационный', 'решение'], ['инновационный', 'решение', 'важно'], ['решение', 'важно', 'синхронизировать'], ['важно', 'синхронизировать', 'усилие'], ['синхронизировать', 'усилие', 'команда'], ['усилие', 'команда', 'достигнуть'], ['команда', 'достигнуть', 'поставить'], ['достигнуть', 'поставить', 'цель'], ['поставить', 'цель', 'каждый'], ['цель', 'каждый', 'сотрудник'], ['каждый', 'сотрудник', 'ключевой'], ['сотрудник', 'ключевой', 'звено'], ['ключевой', 'звено', 'вносить'], ['звено', 'вносить', 'свой'], ['вносить', 'свой', 'вклад'], ['свой', 'вклад', 'успех'], ['вклад', 'успех', 'работа'], ['успех', 'работа', 'компания'], ['работа', 'компания', 'требовать'], ['компания', 'требовать', 'технический'], ['

In [35]:
print(triples_doc_Friends)

[['компания', 'друг', 'каждый'], ['друг', 'каждый', 'встреча'], ['каждый', 'встреча', 'это'], ['встреча', 'это', 'праздник'], ['это', 'праздник', 'делиться'], ['праздник', 'делиться', 'радость'], ['делиться', 'радость', 'смех'], ['радость', 'смех', 'история'], ['смех', 'история', 'вместе'], ['история', 'вместе', 'проводить'], ['вместе', 'проводить', 'время'], ['проводить', 'время', 'природа'], ['время', 'природа', 'готовить'], ['природа', 'готовить', 'вкусный'], ['готовить', 'вкусный', 'блюдо'], ['вкусный', 'блюдо', 'устраивать'], ['блюдо', 'устраивать', 'пикник'], ['устраивать', 'пикник', 'продукт'], ['пикник', 'продукт', 'который'], ['продукт', 'который', 'выбирать'], ['который', 'выбирать', 'любовь'], ['выбирать', 'любовь', 'становиться'], ['любовь', 'становиться', 'часть'], ['становиться', 'часть', 'наш'], ['часть', 'наш', 'общий'], ['наш', 'общий', 'стол'], ['общий', 'стол', 'фрукт'], ['стол', 'фрукт', 'сыр'], ['фрукт', 'сыр', 'каждый'], ['сыр', 'каждый', 'вносить'], ['каждый', 'в

In [36]:
from navec import Navec
from numpy import mean
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances

navec = Navec.load(r'C:\Users\Юлия\OneDrive\Документы\Универ\3 курс\8 триместр\Курсовая\Semantic annotation\files\navec_hudlit_v1_12B_500K_300d_100q.tar')

def get_text_vector(text):
    doc = Doc(text)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    doc.parse_syntax(syntax_parser)
    for token in doc.tokens:
        token.lemmatize(morph_vocab)

    navec_list = []
    for token in doc.tokens:
        lemma = token.lemma.lower()
        if lemma in navec:
            navec_list.append(navec[lemma])

    vector = None
    if len(navec_list) > 0:
        vector = mean(navec_list, axis=0)

    return vector

In [37]:

def get_right_ontology(word, all_ontologies, triplets, metric):
    word_ontologies = all_ontologies[word]

    max_cos = float('-inf') if metric == cosine_similarity else float('inf')
    result_ontology = None

    for ontology in word_ontologies:
        ontology_vector = get_text_vector(ontology)
        for triplet in triplets:
            for lemma in triplet:
                if word != lemma.upper() and lemma.upper() in all_ontologies:
                    lemma_ontologies = all_ontologies[lemma.upper()]

                    for onto in lemma_ontologies:
                        vector = get_text_vector(onto)
                        similarity_score = metric([ontology_vector], [vector])[0][0]
                        if metric == cosine_similarity and similarity_score > max_cos or metric == euclidean_distances and similarity_score < max_cos:
                            max_cos = similarity_score
                            result_ontology = ontology

    return result_ontology

In [38]:
def get_new_ontology_dict(ontology_dict, triples_doc, metric):
    result_ontologies_doc = dict()

    for word, ontologies in ontology_dict.items():

        # if len(ontologies) == 1:
        #     result_ontologies_doc[word] = ontologies[0]

        if len(ontologies) >= 2:
            triples = [triple for triple in triples_doc if word.lower() in triple]
            ontology = get_right_ontology(word, ontology_dict, triples, metric)
            result_ontologies_doc[word] = ontology

    return result_ontologies_doc

In [39]:
result_ontologies_doc_IT = get_new_ontology_dict(ontologies_doc_IT, triples_doc_IT, cosine_similarity)
result_ontologies_doc_IT

{'КОМПАНИЯ': 'компания, осуществляющая свою деятельность в области информационных технологий',
 'РЕШЕНИЕ': 'обдумывать ситуацию, формулируя определённый вывод, делая заключение или вырабатывая план какого-то действия',
 'ЦЕЛЬ': 'заглянуть в будущее, сконцетрировать свое внимание и силы на то, что должно быть достигнуто.',
 'ЗВЕНО': 'составная часть объекта или процесса, воздействие на которую позволяет управлять всем объектом или полным процессом.',
 'ПРОДУКТ': 'компьютерная программа, предназначенная для выполнения определенной задачи, отличной от той, которая связана с работой самого компьютера, обычно предназначенная для использования конечными пользователями',
 'ИТ': 'смелое стремление к чему-либо (обычно высокому, благородному, новому и т. п)'}

In [40]:
result_ontologies_doc_Friends = get_new_ontology_dict(ontologies_doc_Friends, triples_doc_Friends, euclidean_distances)
result_ontologies_doc_Friends

{'КОМПАНИЯ': 'группа лиц, проводящих время совместно',
 'ДЕЛИТЬСЯ': 'доставить радость кому-либо, вызвать сильное положительное чувство в ком-либо',
 'ИСТОРИЯ': 'наука, изучающая факты, тенденции и закономерности развития общества',
 'ВРЕМЯ': 'заняться чем-либо в течении некоторого времени.',
 'ПРОДУКТ': 'компьютерная программа, предназначенная для выполнения определенной задачи, отличной от той, которая связана с работой самого компьютера, обычно предназначенная для использования конечными пользователями',
 'АТМОСФЕРА': 'внесистемная единица измерения давления, численно равная одной килограмм-силе на квадратный сантиметр, что примерно соответствует нормальному атмосферному давлению'}